Group members:


Zhang Fengzhi A0298678X (Responsible for data processing and confidence band filling)


Xie Conghui A0297300J   (Responsible for line animation and slider)


Deng Xinyi A0298862H    (Responsible for framework construction)


Li Jinheng A0299039M    (Responsible for label buttons and adding time intervals)


Chen Shu A0297139N      (Responsible for tail mark and axis modifications)

Reminder: Graphics can't be displayed on html pages, you need to run them locally!

In [11]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [12]:
Global = pd.read_csv("../data/Global Temperature Anomalies.csv")
pi = pd.read_csv("../data/piControl.csv")
force = pd.read_csv("../data/forcings.csv")

Global = Global[Global['Hemisphere'] == 'Global'][["Year", "J-D"]]
Global.columns = ["Year","Observed"]

df = pd.merge(Global, force, on="Year")
df

,Year,Observed,All forcings,Human,Natural,Anthropogenic tropospheric aerosol,Greenhouse gases,Land use,Orbital changes,Ozone,Solar,Volcanic
0,1880,-0.16,287.503107,287.500159,287.462210,287.362800,287.505983,287.457472,287.433307,287.400387,287.399202,287.440784
1,1881,-0.07,287.553577,287.547503,287.502034,287.427485,287.492324,287.404556,287.407178,287.395704,287.417182,287.470855
2,1882,-0.10,287.529591,287.462844,287.492750,287.420378,287.452898,287.438077,287.424884,287.386473,287.447870,287.468017
3,1883,-0.16,287.474062,287.493433,287.385684,287.449032,287.500191,287.457605,287.423441,287.399029,287.435342,287.402655
4,1884,-0.28,287.147241,287.562729,287.077431,287.373120,287.487975,287.440270,287.444116,287.395991,287.474246,287.070387
...,...,...,...,...,...,...,...,...,...,...,...,...
121,2001,0.54,288.320929,288.184239,287.526614,286.968754,288.552342,287.323666,287.389576,287.518246,287.472521,287.499175
122,2002,0.63,288.335254,288.218673,287.545374,287.022696,288.594759,287.361851,287.399834,287.545542,287.477142,287.504434
123,2003,0.62,288.348285,288.259768,287.534229,287.033548,288.595026,287.345990,287.460698,287.503628,287.479919,287.515221
124,2004,0.54,288.382879,288.269479,287.559768,287.020872,288.615446,287.320629,287.419639,287.500390,287.491381,287.534006


In [13]:
# Calculation of 1880-2005 length
years = len(df)

# Numerical modelling started in 1850, so 30 years of backward adjustment
pi = pi.iloc[30:years+30,:].reset_index(drop=True)

In [14]:
df.iloc[:,2:] = df.iloc[:,2:].sub(pi['Temp (K)']+0.1,axis=0)
df

,Year,Observed,All forcings,Human,Natural,Anthropogenic tropospheric aerosol,Greenhouse gases,Land use,Orbital changes,Ozone,Solar,Volcanic
0,1880,-0.16,-0.026391,-0.029339,-0.067289,-0.166699,-0.023515,-0.072026,-0.096192,-0.129111,-0.130297,-0.088714
1,1881,-0.07,-0.063298,-0.069372,-0.114842,-0.189390,-0.124551,-0.212319,-0.209698,-0.221171,-0.199693,-0.146020
2,1882,-0.10,-0.084821,-0.151568,-0.121662,-0.194034,-0.161514,-0.176335,-0.189528,-0.227939,-0.166541,-0.146395
3,1883,-0.16,-0.047180,-0.027810,-0.135558,-0.072210,-0.021051,-0.063637,-0.097801,-0.122213,-0.085900,-0.118587
4,1884,-0.28,-0.540430,-0.124942,-0.610240,-0.314550,-0.199695,-0.247401,-0.243554,-0.291679,-0.213424,-0.617284
...,...,...,...,...,...,...,...,...,...,...,...,...
121,2001,0.54,0.709654,0.572963,-0.084662,-0.642521,0.941067,-0.287609,-0.221699,-0.093030,-0.138754,-0.112100
122,2002,0.63,0.777520,0.660939,-0.012360,-0.535038,1.037025,-0.195883,-0.157900,-0.012192,-0.080592,-0.053300
123,2003,0.62,0.793813,0.705295,-0.020243,-0.520925,1.040554,-0.208483,-0.093774,-0.050844,-0.074553,-0.039252
124,2004,0.54,0.868099,0.754699,0.044987,-0.493909,1.100666,-0.194152,-0.095141,-0.014390,-0.023399,0.019225


In [15]:
df = df[['Year','Observed','All forcings','Anthropogenic tropospheric aerosol','Greenhouse gases','Land use','Orbital changes','Ozone','Solar','Volcanic']]
df

,Year,Observed,All forcings,Anthropogenic tropospheric aerosol,Greenhouse gases,Land use,Orbital changes,Ozone,Solar,Volcanic
0,1880,-0.16,-0.026391,-0.166699,-0.023515,-0.072026,-0.096192,-0.129111,-0.130297,-0.088714
1,1881,-0.07,-0.063298,-0.189390,-0.124551,-0.212319,-0.209698,-0.221171,-0.199693,-0.146020
2,1882,-0.10,-0.084821,-0.194034,-0.161514,-0.176335,-0.189528,-0.227939,-0.166541,-0.146395
3,1883,-0.16,-0.047180,-0.072210,-0.021051,-0.063637,-0.097801,-0.122213,-0.085900,-0.118587
4,1884,-0.28,-0.540430,-0.314550,-0.199695,-0.247401,-0.243554,-0.291679,-0.213424,-0.617284
...,...,...,...,...,...,...,...,...,...,...
121,2001,0.54,0.709654,-0.642521,0.941067,-0.287609,-0.221699,-0.093030,-0.138754,-0.112100
122,2002,0.63,0.777520,-0.535038,1.037025,-0.195883,-0.157900,-0.012192,-0.080592,-0.053300
123,2003,0.62,0.793813,-0.520925,1.040554,-0.208483,-0.093774,-0.050844,-0.074553,-0.039252
124,2004,0.54,0.868099,-0.493909,1.100666,-0.194152,-0.095141,-0.014390,-0.023399,0.019225


In [16]:
def bootstrap_ci(data, num_bootstrap_samples=1000, ci=95, width_multiplier=1.0):
    bootstrap_means = [np.mean(np.random.choice(data, size=len(data), replace=True)) 
                       for _ in range(num_bootstrap_samples)]
    
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    
    lower_bound = np.percentile(bootstrap_means, lower_percentile)
    upper_bound = np.percentile(bootstrap_means, upper_percentile)
    
    mean_estimate = np.mean(bootstrap_means)
    lower_bound_adjusted = mean_estimate - (mean_estimate - lower_bound) * width_multiplier
    upper_bound_adjusted = mean_estimate + (upper_bound - mean_estimate) * width_multiplier
    
    return lower_bound_adjusted, upper_bound_adjusted

def calculate_bootstrap_ci(df, columns, window_size=1, num_bootstrap_samples=1000, ci=95, width_multiplier=1.0):
    df_bound = df.copy()

    for column in columns:
        ci_lower = []
        ci_upper = []

        column_data = df[column]

        for i in range(len(df)):
            # Extract the data within the rolling window
            window_data = column_data[max(0, i - window_size):min(len(column_data), i + window_size + 1)]

            # Calculate bootstrap confidence intervals
            lower, upper = bootstrap_ci(window_data, num_bootstrap_samples, ci, width_multiplier)
            ci_lower.append(lower)
            ci_upper.append(upper)

        # Add the lower and upper bounds to the DataFrame
        df_bound[f'{column} Lower'] = ci_lower
        df_bound[f'{column} Upper'] = ci_upper

    return df_bound



In [17]:
# Generate data every 3,5,10 years
df3 = df.groupby(df['Year'] // 3 * 3,as_index=False).mean()
df5 = df.groupby(df['Year'] // 5 * 5,as_index=False).mean()
df10 = df.groupby(df['Year'] // 10 * 10,as_index=False).mean()

In [18]:
df_bound = calculate_bootstrap_ci(df, columns=['All forcings'])
df_bound3 = calculate_bootstrap_ci(df3, columns=['All forcings'])
df_bound5 = calculate_bootstrap_ci(df5, columns=['All forcings'])
df_bound10 = calculate_bootstrap_ci(df10, columns=['All forcings'])

In [19]:
# Average 1880-1910
Mean = df['Observed'].iloc[:30].mean()
Mean

-0.24699999999999994

In [21]:
app = Dash(__name__)

labels = [ 
    'Anthropogenic tropospheric aerosol', 'Greenhouse gases', 
    'Land use', 'Orbital changes', 'Ozone', 'Solar', 'Volcanic'
]
colors = [ '#34495e', 
    '#e67e22', '#f1c40f', '#95a5a6', '#d35400', '#2ecc71', '#8e44ad'
]


app.layout = html.Div([
        dcc.Dropdown(
        id='interval-dropdown',
        options=[
            {'label': 'Every year', 'value': 'df'},
            {'label': 'Average Every 3 Years', 'value': 'df3'},
            {'label': 'Average Every 5 Years', 'value': 'df5'},
            {'label': 'Average Every 10 Years', 'value': 'df10'}
        ],
        value='df', 
        style={'width': '50%', 'margin': 'auto'}
    ),
    dcc.Graph(
        id='line-chart',
        config={'displayModeBar': False},
        style={'backgroundColor': 'white', 'width': '70%', 'height': '80vh', 'margin': 'auto'}
    ),
    html.Div([
        html.Button(label, id={'type': 'label-button', 'index': label}, n_clicks=0, style={
            'backgroundColor': color,
            'color': 'white',
            'padding': '5px 10px',
            'border-radius': '5px',
            'margin': '3px',
            'text-align': 'center',
            'font-size': '12px',
            'border': 'none',
            'cursor': 'pointer'
        }) for label, color in zip(labels, colors)
    ], style={
        'text-align': 'center',
        'display': 'flex',
        'flex-direction': 'row',
        'flex-wrap': 'wrap',
        'justify-content': 'center',
        'align-items': 'center',
        'margin-top': '20px'
    })
])

dataframes = {'df': df, 'df3': df3, 'df5': df5, 'df10': df10}
dataframes_bound = {'df': df_bound, 'df3': df_bound3, 'df5': df_bound5, 'df10': df_bound10}


@app.callback(
    Output('line-chart', 'figure'),
    [Input({'type': 'label-button', 'index': label}, 'n_clicks') for label in labels] +
    [Input('interval-dropdown', 'value')]
)

def update_chart(*args):
    selected_df_key = args[-1]
    selected_df = dataframes[selected_df_key]
    df_confidence = dataframes_bound[selected_df_key]
    
    fig = go.Figure()
    frames = []
    
    observed_trace = go.Scatter(
        x=selected_df['Year'],
        y=selected_df['Observed'],
        mode='lines+text',
        name='Observed',
        line=dict(width=2, color='black'),
        text=[None] * (len(selected_df['Year']) - 1) + ['Observed'],
        textposition='top right',
        showlegend=False 
    )
    
    all_forcings_trace = go.Scatter(
        x=selected_df['Year'],
        y=selected_df['All forcings'],
        mode='lines+text',
        name='All forcings',
        line=dict(width=2, color='red'),
        text=[None] * (len(selected_df['Year']) - 1) + ['All forcings'],
        textposition='top right',
        showlegend=False 
    )
    

    all_forcings_upper = go.Scatter(
        x=df_confidence['Year'],
        y=df_confidence['All forcings Upper'], 
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    )
    all_forcings_lower = go.Scatter(
        x=df_confidence['Year'],
        y=df_confidence['All forcings Lower'], 
        mode='lines',
        fill='tonexty',
        fillcolor='rgba(255, 0, 0, 0.2)', 
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    )
    
    fig.add_trace(observed_trace)
    fig.add_trace(all_forcings_trace)
    fig.add_trace(all_forcings_upper)
    fig.add_trace(all_forcings_lower)

    active_traces = []
    for i, (label, clicks) in enumerate(zip(labels, args[:-1])):
        if clicks % 2 == 1:  
            active_traces.append(go.Scatter(
                x=selected_df['Year'],
                y=selected_df[label],
                mode='lines+text',
                name=label,
                line=dict(width=2, color=colors[i]),
                text=[None] * (len(selected_df['Year']) - 1) + [label],  
                textposition='top right',
                showlegend=False
            ))

    fig.add_traces(active_traces)

    for frame_idx in range(len(selected_df['Year'])):
        frame_data = [observed_trace,
            all_forcings_trace,
            all_forcings_upper,
            all_forcings_lower
        ] + [
            go.Scatter(
                x=selected_df['Year'][:frame_idx + 1],
                y=selected_df[trace.name][:frame_idx + 1],
                mode='lines+text',
                name=trace.name,
                line=dict(width=2, color=trace.line.color),
                text=[None] * frame_idx + [trace.name],
                textposition='top right',
                showlegend=False
            ) for trace in active_traces
        ]
        
        frames.append(go.Frame(
            data=frame_data,
            name=str(selected_df['Year'][frame_idx])
        ))

# def update_chart(*args):
#     selected_df_key = args[-1]
#     selected_df = dataframes[selected_df_key]
    
#     fig = go.Figure()
#     frames = []
    
#     observed_trace = go.Scatter(
#         x=selected_df['Year'],
#         y=selected_df['Observed'],
#         mode='lines',
#         name='Observed',
#         line=dict(width=2, color='black'),
#         showlegend=False 
#     )
#     all_forcings_trace = go.Scatter(
#         x=selected_df['Year'],
#         y=selected_df['All forcings'],
#         mode='lines',
#         name='All forcings',
#         line=dict(width=2, color='red'),
#         showlegend=False 
#     )
    
#     all_forcings_upper = go.Scatter(
#         x=df_bound['Year'],
#         y=df_bound['All forcings Upper'],  
#         mode='lines',
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     )
#     all_forcings_lower = go.Scatter(
#         x=df_bound['Year'],
#         y=df_bound['All forcings Lower'], 
#         mode='lines',
#         fill='tonexty',  
#         fillcolor='rgba(255, 0, 0, 0.2)', 
#         line=dict(width=0),
#         showlegend=False,
#         hoverinfo='skip'
#     )
    
#     fig.add_trace(observed_trace)
#     fig.add_trace(all_forcings_trace)
#     fig.add_trace(all_forcings_upper)
#     fig.add_trace(all_forcings_lower)
    
#     active_traces = []
#     for i, (label, clicks) in enumerate(zip(labels, args[:-1])):
#         if clicks % 2 == 1:  # Only add trace if clicks are odd
#             active_traces.append(go.Scatter(
#                 x=selected_df['Year'],
#                 y=selected_df[label],
#                 mode='lines',
#                 name=label,
#                 line=dict(width=2, color=colors[i]),
#                 showlegend=False
#             ))

#     fig.add_traces(active_traces)

#     for frame_idx in range(len(selected_df['Year'])):
#         frame_data = [
#             observed_trace,
#             all_forcings_trace,
#             all_forcings_upper,
#             all_forcings_lower
#         ] + [
#             go.Scatter(
#                 x=selected_df['Year'][:frame_idx + 1],
#                 y=selected_df[trace.name][:frame_idx + 1],
#                 mode='lines',
#                 name=trace.name,
#                 line=dict(width=2, color=trace.line.color),
#                 showlegend=False
#             ) for trace in active_traces
#         ]
        
#         frames.append(go.Frame(
#             data=frame_data,
#             name=str(selected_df['Year'][frame_idx])
#         ))

    fig.update_layout(
        template='plotly_white',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(
            visible=False,
            range=[selected_df['Year'].min(), selected_df['Year'].max()+30],
            tickvals=[df['Year'].min(), df['Year'].max()],
            ticktext=["1880", "2005"],
            tickangle=45,
        ),
        yaxis=dict(
            range=[Mean - 2, Mean + 2],
            showticklabels=True,
            showline=True,
            linecolor='gray',
            ticks="outside",
            ticklen=4,
            tickwidth=1,
            tickcolor='gray',
            tickvals=[Mean - 2, Mean, Mean + 2],
            ticktext=["-2℉", "1880-1910\nAverage", "+2°C"],
            tickangle=0,
            linewidth=2
            ),
        updatemenus=[{
            "type": "buttons",
            "showactive": False,
            "x": -0.1,
            "y": 0.2,
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 50, "redraw": True}, "fromcurrent": True}],
                    "label": "play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate",
                                    "transition": {"duration": 0}}], 
                    "label": "pause",
                    "method": "animate"
                }
            ]
        }],
        sliders=[{
            "y": -0.05,
            "steps": [
                {"args": [[str(year)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                 "label": str(year),
                 "method": "animate"} for year in selected_df['Year']
            ],
            "active": 0,
        }],
        annotations=[
            dict(
                x=0.01, y=Mean + 2,
                xref="paper", yref="y",
                text="Hotter", showarrow=False,
                font=dict(size=12, color="black"),
            ),
            dict(
                x=0.01, y=Mean - 2,
                xref="paper", yref="y",
                text="Colder", showarrow=False,
                font=dict(size=12, color="black"),
            ),
            dict(
                x=0.5, y=-0.2,
                xref="paper", yref="paper",
                text="1880-2005", showarrow=False,
                font=dict(size=25, color="gray"),
            )
        ],
        shapes=[
            dict(
                type='line',
                x0=df['Year'].min(), y0=Mean,
                x1=df['Year'].max(), y1=Mean,
                line=dict(color='gray', width=2)
            ) 
        ],
    )

    fig.update(frames=frames)
    return fig


In [ ]:
## !!!!! Please Read:
## Click on the button to show the corresponding line, click again to disappear, support multiple lines playback at the same time
## Try to use slide track, the tail markers will blink when the play button is playing.

if __name__ == '__main__':
    app.run_server(debug=True, port=8063)

We acknowledge the use of ChatGPT to debug code that was subsequently included in modified form in my report (https://chatgpt.com/share/6731c4a5-c804-8004-91b9-ccb8a51e7929). We entered the following prompt(s) on November 11, 2024: How to animate data.......

We acknowledge the use of ChatGPT to debug code that was subsequently included in modified form in my report (https://chatgpt.com/share/67320d24-3aa0-8002-b8ce-105d7f840bfe). We entered the following prompt(s) on November 11, 2024: Using dash and plotly for diagrams, how to add a drop down menu to show other diagrams. How to add a label button below the chart that can be interactively clicked to display the line......